In [12]:
import googlemaps, requests, pandas as pd, os
from dotenv import load_dotenv
load_dotenv()  # load environment vars
gmaps = googlemaps.Client(key=os.getenv("GOOGLE_MAP_API_KEY"))

In [13]:
yt_data = pd.read_csv("/Users/discovery/Desktop/testing_rag/yt/data/youtube_videos.csv")
yt_data.head(2)

,Title,Link,Description,PublishedAt,Links,video_id,transcript
0,I Reviewed AUSTRIA'S FIRST EVER 3 MICHELIN STA...,https://www.youtube.com/watch?v=ZJMQ7dNT9YQ,Today I’m taking you to Austria’s first 3-star...,2025-05-30T16:00:53Z,['https://www.falstaff.com/ch/rezepte/kochen/b...,ZJMQ7dNT9YQ,"Error: no element found: line 1, column 0"
1,I Ate at EVERY MICHELIN Starred Restaurant in ...,https://www.youtube.com/watch?v=CHwWZi_CP2Y,Here it is. Our biggest video ever.\nI ate at ...,2025-05-21T16:00:17Z,['https://www.instagram.com/alexandertheguest'...,CHwWZi_CP2Y,"Error: no element found: line 1, column 0"


In [14]:
data = pd.read_csv("/Users/discovery/Desktop/testing_rag/yt/data/youtube_videos_info.csv",encoding='cp1252')
data.head(2)

,Title,Link,Description,Publish Date,Location,Restaurant Visited
0,I Reviewed AUSTRIA'S FIRST EVER 3 MICHELIN STA...,https://www.youtube.com/watch?v=ZJMQ7dNT9YQ,The video features a detailed review of Austri...,5/30/25,Vienna,Amador
1,The RESTAURANT That Made HISTORY for Me (105-Y...,https://www.youtube.com/watch?v=dxQ_o1Z5yB8,Join Alexander as he explores the renowned Ame...,5/1/25,San Sebastian,Amelia


In [15]:
merged = pd.merge(data, yt_data[['Link', 'PublishedAt']], on='Link', how='left')
merged.head()

,Title,Link,Description,Publish Date,Location,Restaurant Visited,PublishedAt
0,I Reviewed AUSTRIA'S FIRST EVER 3 MICHELIN STA...,https://www.youtube.com/watch?v=ZJMQ7dNT9YQ,The video features a detailed review of Austri...,5/30/25,Vienna,Amador,2025-05-30T16:00:53Z
1,The RESTAURANT That Made HISTORY for Me (105-Y...,https://www.youtube.com/watch?v=dxQ_o1Z5yB8,Join Alexander as he explores the renowned Ame...,5/1/25,San Sebastian,Amelia,2025-05-01T16:03:42Z
2,Dining at Thailandâ€™s ONLY 3 MICHELIN STAR Re...,https://www.youtube.com/watch?v=pnWtFQGUkFY,Iâ€™m gonna take you to the first and only 3 M...,##############################################...,"Thailand, Bangkok",SORN,2025-04-17T16:01:17Z
3,I Tried the $560 MENU at One of Asiaâ€™s MOST ...,https://www.youtube.com/watch?v=p2HMymAIx64,"Today, I'm taking you to one of Asia's most lu...",##############################################...,Macau,Jo‘l Robuchon au D™me,2025-04-03T16:00:21Z
4,I Stayed at the #1 HOTEL IN THE WORLD,https://www.youtube.com/watch?v=6Jm68betmCI,"In this video, the host explores the worldâ€™s...",3/24/25,"Bangkok, Thailand",Capella Bangkok,2025-03-24T17:01:02Z


In [16]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Title               114 non-null    object
 1   Link                114 non-null    object
 2   Description         113 non-null    object
 3   Publish Date        114 non-null    object
 4   Location            114 non-null    object
 5   Restaurant Visited  114 non-null    object
 6   PublishedAt         114 non-null    object
dtypes: object(7)
memory usage: 6.4+ KB


In [19]:
def fetch_lat_lng_for_restaurant(restaurant, location):
    location_result = gmaps.geocode(f"{restaurant}, {location}")
    if location_result:
        return location_result[0]['geometry']['location']['lat'], location_result[0]['geometry']['location']['lng']
    return None, None

In [20]:
merged[['Lat', 'Lng']] = merged.apply(lambda row: pd.Series(fetch_lat_lng_for_restaurant(row['Restaurant Visited'], row['Location'])), axis=1)
merged.head()

,Title,Link,Description,Publish Date,Location,Restaurant Visited,PublishedAt,Lat,Lng
0,I Reviewed AUSTRIA'S FIRST EVER 3 MICHELIN STA...,https://www.youtube.com/watch?v=ZJMQ7dNT9YQ,The video features a detailed review of Austri...,5/30/25,Vienna,Amador,2025-05-30T16:00:53Z,48.253930,16.358612
1,The RESTAURANT That Made HISTORY for Me (105-Y...,https://www.youtube.com/watch?v=dxQ_o1Z5yB8,Join Alexander as he explores the renowned Ame...,5/1/25,San Sebastian,Amelia,2025-05-01T16:03:42Z,43.316923,-1.986149
2,Dining at Thailandâ€™s ONLY 3 MICHELIN STAR Re...,https://www.youtube.com/watch?v=pnWtFQGUkFY,Iâ€™m gonna take you to the first and only 3 M...,##############################################...,"Thailand, Bangkok",SORN,2025-04-17T16:01:17Z,13.723100,100.568525
3,I Tried the $560 MENU at One of Asiaâ€™s MOST ...,https://www.youtube.com/watch?v=p2HMymAIx64,"Today, I'm taking you to one of Asia's most lu...",##############################################...,Macau,Jo‘l Robuchon au D™me,2025-04-03T16:00:21Z,22.198745,113.543873
4,I Stayed at the #1 HOTEL IN THE WORLD,https://www.youtube.com/watch?v=6Jm68betmCI,"In this video, the host explores the worldâ€™s...",3/24/25,"Bangkok, Thailand",Capella Bangkok,2025-03-24T17:01:02Z,13.713105,100.511022


In [ ]:
# for index, row in data.iterrows():
    
#     fetch_lat_lng_for_restaurant(row['Restaurant Visited'], row['Location'])

#     if location_result:
#         lat = location_result[0]['geometry']['location']['lat']
#         lng = location_result[0]['geometry']['location']['lng']
#         print(f"Latitude: {lat}, Longitude: {lng}")
#     else:
#         print("Restaurant not found or geocoding failed.")
    
    # Name of the restaurant
    # Date of video published 
    # youtube link
    # map link
    # Address
    # Description
    

Latitude: 48.2539302, Longitude: 16.3586121
Latitude: 43.3169229, Longitude: -1.9861492
Latitude: 13.7231004, Longitude: 100.5685251
Latitude: 22.198745, Longitude: 113.543873
Latitude: 13.7131047, Longitude: 100.5110225
Latitude: 48.1447926, Longitude: 11.5629595
Latitude: 45.4515503, Longitude: 9.1616272
Latitude: 63.21441963281464, Longitude: 10.8093569414146
Latitude: 25.132137, Longitude: 55.11782789999999
Latitude: 49.937877, Longitude: 6.81114
Latitude: 1.2818449, Longitude: 103.8505325
Latitude: 51.5203787, Longitude: -0.1365322
Latitude: 46.7927308, Longitude: 11.9491536
Latitude: -8.5422753, Longitude: 115.2573302
Latitude: 48.8583698, Longitude: 2.2944833
Latitude: 51.5123709, Longitude: -0.1155466
Latitude: 43.71946310000001, Longitude: 13.2208132
Latitude: 55.74829889999999, Longitude: 12.5412079
Latitude: 51.516655, Longitude: -0.200063
Latitude: 22.281517, Longitude: 114.1587556


KeyboardInterrupt: 

In [23]:
df = merged

In [ ]:
merged_data

,Title,Link,Description,Publish Date,Location,Restaurant Visited,PublishedAt,Lat,Lng
0,I Reviewed AUSTRIA'S FIRST EVER 3 MICHELIN STA...,https://www.youtube.com/watch?v=ZJMQ7dNT9YQ,The video features a detailed review of Austri...,5/30/25,Vienna,Amador,2025-05-30T16:00:53Z,48.253930,16.358612
1,The RESTAURANT That Made HISTORY for Me (105-Y...,https://www.youtube.com/watch?v=dxQ_o1Z5yB8,Join Alexander as he explores the renowned Ame...,5/1/25,San Sebastian,Amelia,2025-05-01T16:03:42Z,43.316923,-1.986149
2,Dining at Thailandâ€™s ONLY 3 MICHELIN STAR Re...,https://www.youtube.com/watch?v=pnWtFQGUkFY,Iâ€™m gonna take you to the first and only 3 M...,##############################################...,"Thailand, Bangkok",SORN,2025-04-17T16:01:17Z,13.723100,100.568525
3,I Tried the $560 MENU at One of Asiaâ€™s MOST ...,https://www.youtube.com/watch?v=p2HMymAIx64,"Today, I'm taking you to one of Asia's most lu...",##############################################...,Macau,Jo‘l Robuchon au D™me,2025-04-03T16:00:21Z,22.198745,113.543873
4,I Stayed at the #1 HOTEL IN THE WORLD,https://www.youtube.com/watch?v=6Jm68betmCI,"In this video, the host explores the worldâ€™s...",3/24/25,"Bangkok, Thailand",Capella Bangkok,2025-03-24T17:01:02Z,13.713105,100.511022
...,...,...,...,...,...,...,...,...,...
109,NOTHING is what it seems in this restaurant - ...,https://www.youtube.com/watch?v=Sk9xOktLCqk,"Alexander, co-owner of a Michelin star restaur...",10/3/23,"Bray, United Kingdom",The Fat Duck (Heston Blumenthal),2023-03-10T15:00:19Z,51.507914,-0.701748
110,The founder and head chef is Enrico Crippa.Ê H...,https://www.youtube.com/watch?v=4tdrMksmbZ0,"Join Alexander, a Michelin-starred restaurant ...",5/31/23,Italy,Piazza Duomo Chef Crippa,2023-05-31T15:11:02Z,44.700584,8.035861
111,"Today, we go to Monaco, home to our restaurant...",https://www.youtube.com/watch?v=-RAmiqe2neU,"Today, we go to Monaco, home to our restaurant...",5/5/23,MONACO,Le Louis XV - Alain Ducasse at L'H™tel de Pari...,2023-05-05T15:34:46Z,43.739074,7.426736
112,"Û2,222 Lunch with the BEST SERVICE EVER - Mira...",https://www.youtube.com/watch?v=lsZBiroyuCY,"Join Alexander, a Michelin-starred restaurant ...",4/7/23,Frence Riviera,Mirazur,2023-04-07T14:41:59Z,43.785839,7.527915


In [42]:
df = df.dropna(subset=['Lat', 'Lng']).reset_index(drop=True)
df

,Title,Link,Description,Publish Date,Location,Restaurant Visited,PublishedAt,Lat,Lng
0,I Reviewed AUSTRIA'S FIRST EVER 3 MICHELIN STA...,https://www.youtube.com/watch?v=ZJMQ7dNT9YQ,The video features a detailed review of Austri...,5/30/25,Vienna,Amador,2025-05-30T16:00:53Z,48.253930,16.358612
1,The RESTAURANT That Made HISTORY for Me (105-Y...,https://www.youtube.com/watch?v=dxQ_o1Z5yB8,Join Alexander as he explores the renowned Ame...,5/1/25,San Sebastian,Amelia,2025-05-01T16:03:42Z,43.316923,-1.986149
2,Dining at Thailandâ€™s ONLY 3 MICHELIN STAR Re...,https://www.youtube.com/watch?v=pnWtFQGUkFY,Iâ€™m gonna take you to the first and only 3 M...,##############################################...,"Thailand, Bangkok",SORN,2025-04-17T16:01:17Z,13.723100,100.568525
3,I Tried the $560 MENU at One of Asiaâ€™s MOST ...,https://www.youtube.com/watch?v=p2HMymAIx64,"Today, I'm taking you to one of Asia's most lu...",##############################################...,Macau,Jo‘l Robuchon au D™me,2025-04-03T16:00:21Z,22.198745,113.543873
4,I Stayed at the #1 HOTEL IN THE WORLD,https://www.youtube.com/watch?v=6Jm68betmCI,"In this video, the host explores the worldâ€™s...",3/24/25,"Bangkok, Thailand",Capella Bangkok,2025-03-24T17:01:02Z,13.713105,100.511022
...,...,...,...,...,...,...,...,...,...
108,NOTHING is what it seems in this restaurant - ...,https://www.youtube.com/watch?v=Sk9xOktLCqk,"Alexander, co-owner of a Michelin star restaur...",10/3/23,"Bray, United Kingdom",The Fat Duck (Heston Blumenthal),2023-03-10T15:00:19Z,51.507914,-0.701748
109,The founder and head chef is Enrico Crippa.Ê H...,https://www.youtube.com/watch?v=4tdrMksmbZ0,"Join Alexander, a Michelin-starred restaurant ...",5/31/23,Italy,Piazza Duomo Chef Crippa,2023-05-31T15:11:02Z,44.700584,8.035861
110,"Today, we go to Monaco, home to our restaurant...",https://www.youtube.com/watch?v=-RAmiqe2neU,"Today, we go to Monaco, home to our restaurant...",5/5/23,MONACO,Le Louis XV - Alain Ducasse at L'H™tel de Pari...,2023-05-05T15:34:46Z,43.739074,7.426736
111,"Û2,222 Lunch with the BEST SERVICE EVER - Mira...",https://www.youtube.com/watch?v=lsZBiroyuCY,"Join Alexander, a Michelin-starred restaurant ...",4/7/23,Frence Riviera,Mirazur,2023-04-07T14:41:59Z,43.785839,7.527915


In [51]:
df['PublishedAt'] = [date[:10] for date in df['PublishedAt']]

In [65]:
from folium import Map, FeatureGroup, Marker, Icon
from folium.plugins import Search

In [68]:
import folium
center_lat = df['Lat'].mean()
center_lng = df['Lng'].mean()
m = folium.Map(location=[center_lat+10, center_lng-10], zoom_start=5)
fg = FeatureGroup().add_to(m)

for _, row in df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    restaurant = row['Restaurant Visited']
    date = row['PublishedAt']
    youtube_url = row['Link']
    address = row['Location']
    description = row['Description']
    
    html = f"""
    <b>Restaurant:</b> {restaurant}<br>
    <b>Publish Date:</b> {date}<br>
    <b>Address:</b> {address}<br>
    <a href="{youtube_url}" target="_blank">
      <i class="fa fa-youtube-play"></i>
    </a>
    <a href="https://www.google.com/maps/search/?api=1&query={lat},{lng}"
       target="_blank">
      Google Maps: {restaurant}
    </a>
    """
    iframe = folium.IFrame(html, width=300, height=200)
    popup = folium.Popup(iframe, max_width=300)
    icon = folium.Icon(prefix='fa', icon='youtube-play', color='red')
    title = folium.Popup(title=restaurant, max_width=300)
    folium.Marker(location=[lat, lng], popup=popup, icon=icon).add_to(fg)

Search(fg, search_label="Restaurant").add_to(m)
m.save('/Users/discovery/Desktop/testing_rag/yt/data/restaurant_map_all.html')
print("Map written to /Users/discovery/Desktop/testing_rag/yt/data/restaurant_map_all.html")

Map written to /Users/discovery/Desktop/testing_rag/yt/data/restaurant_map_all.html


In [39]:
df.isna().sum()

Title                 0
Link                  0
Description           1
Publish Date          0
Location              0
Restaurant Visited    0
PublishedAt           0
Lat                   1
Lng                   1
dtype: int64

In [41]:
rows_with_nans = df[df.isna().any(axis=1)]
rows_with_nans

,Title,Link,Description,Publish Date,Location,Restaurant Visited,PublishedAt,Lat,Lng
29,Why Is CAVIAR So LUXURIOUS And EXPENSIVE?,https://www.youtube.com/watch?v=AITrAK_mGQQ,In this special episode titled 'Alexander The ...,6/23/24,Not clearly identified,42restaurant,2024-06-23T16:00:15Z,NaN,NaN
73,TrÃ©sind is considered one of the best Indian ...,https://www.youtube.com/watch?v=NXro-2NRuaw,NaN,##############################################...,Dubai,TrŽsind,2023-03-08T21:20:30Z,25.204849,55.270783


In [ ]:
import folium

# 1. Define your data attributes
restaurant_name = "Benu"
place_name = "San Francisco"
lat = 37.7855007
lng = -122.3990397

# 2. Create a map centered on your coordinates
#    zoom_start determines the initial zoom level.
m = folium.Map(location=[lat, lng], zoom_start=15)

# 3. Create the HTML content for the hover tooltip
#    Using an f-string with HTML tags makes the output look clean.
tooltip_html = f"""
<b>Restaurant:</b> {restaurant_name}<br>
<b>City:</b> {place_name}<br>
<b>Latitude:</b> {lat}<br>
<b>Longitude:</b> {lng}
"""

# 4. Add a marker to the map with the tooltip
folium.Marker(
    location=[lat, lng],
    tooltip=tooltip_html
).add_to(m)

# 5. Save the map to an HTML file
m.save("/Users/discovery/Desktop/testing_rag/yt/restaurant_map.html")

print("Map has been saved to restaurant_map.html")

In [ ]:
restaurant_name = "Benu"
place_name = "San Francisco"
address = f"{restaurant_name}, {place_name}"
location_result = gmaps.geocode(address)

if location_result:
    lat = location_result[0]['geometry']['location']['lat']
    lng = location_result[0]['geometry']['location']['lng']
    print(f"Latitude: {lat}, Longitude: {lng}")
else:
    print("Restaurant not found or geocoding failed.")

Latitude: 37.7855007, Longitude: -122.3990397


In [8]:
restaurant_name = "Benu"
place_name = "San Francisco"
lat, lng

(37.7855007, -122.3990397)

In [4]:
import folium

# 1. Define your data attributes
restaurant_name = "Benu"
place_name = "San Francisco"
lat = 37.7855007
lng = -122.3990397

# 2. Create a map centered on your coordinates
#    zoom_start determines the initial zoom level.
m = folium.Map(location=[lat, lng], zoom_start=15)

# 3. Create the HTML content for the hover tooltip
#    Using an f-string with HTML tags makes the output look clean.
tooltip_html = f"""
<b>Restaurant:</b> {restaurant_name}<br>
<b>City:</b> {place_name}<br>
<b>Latitude:</b> {lat}<br>
<b>Longitude:</b> {lng}
"""

# 4. Add a marker to the map with the tooltip
folium.Marker(
    location=[lat, lng],
    tooltip=tooltip_html
).add_to(m)

# 5. Save the map to an HTML file
m.save("/Users/discovery/Desktop/testing_rag/yt/restaurant_map.html")

print("Map has been saved to restaurant_map.html")

Map has been saved to restaurant_map.html
